In [1]:
import numpy as np
import os
import torch

from glob import glob
from skimage import io
from tensorboardX import SummaryWriter
from torch.utils.data.dataloader import DataLoader
from torch import nn
from tqdm import tqdm
from time import time

from src.batch_generator import BatchGenerator, WraptorDataLoader
from src.command import Command
from src.models import LocalTransferNet, GlobalTransferer, RefinementNet
from src.train_utils import (draw_scalar_value, draw_images, rgb2gray, infer_batch,
                             inference_all_test_videos, load_all_test_videos, calculate_psnr_result, frame_to_tensor)


from color_utils import rgb2lab_torch, lab2rgb_torch
from skimage.color import rgb2lab, lab2rgb
from collections import defaultdict

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']='0, 2'

experiment_dir = "experiments/train_refinement_onepass_six_frames_loss_pretrainetlocal_amsgrad"
logs_directory = os.path.join(experiment_dir, "logs")
save_test_output_videos_dir =  os.path.join(experiment_dir,  "results_test_videos")
checkpoints_folder =  os.path.join(experiment_dir, "checkpoints")

board_port = 6103
board_timeout=24*60*60
batch_size = 4
img_shape = (128, 128, 3)
val_num = 5
iteration_num = 5000
dataset_dir = "datasets/Davis480/480p"
learning_rate = 1e-3 / 5
verbose_every_it = 10
save_models_every_it = 100

test_every_it = 100

In [3]:
test_videos_folders = sorted(glob(dataset_dir + "/*"))[-val_num:]

In [4]:
writer = SummaryWriter(logs_directory) 
board = Command('tensorboard --logdir=run1:{} --port {}'.format(logs_directory, board_port))
board.run()

In [5]:
dir_paths = [experiment_dir, logs_directory, checkpoints_folder, save_test_output_videos_dir]
for path in dir_paths:
    try:
        os.mkdir(path)
    except:
        pass

In [6]:
train_videos = load_all_test_videos(sorted(glob(dataset_dir + "/*"))[:-val_num], resize_size=img_shape)
val_videos = load_all_test_videos(sorted(glob(dataset_dir + "/*"))[-val_num:], resize_size=img_shape)

/opt/conda/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [7]:
local_transfer_net = nn.DataParallel(LocalTransferNet().double().cuda())
global_transferer = GlobalTransferer()
refinement_net = nn.DataParallel(RefinementNet().double().cuda())

In [8]:
local_transfer_net.load_state_dict(torch.load("experiments/pretrain_local_several_frames_back_highlr_amsgrad/checkpoints/7800/local_net_state_dict"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [9]:
pretrain_local = False
end2end = False

In [10]:
loss_function = torch.nn.L1Loss() #torch.nn.MSELoss() or torch.nn.L1Loss()
if pretrain_local:
    optimizer = torch.optim.Adam(list(local_transfer_net.parameters()), 
                                 lr=learning_rate, amsgrad=True)
else:
    if end2end:
        optimizer = torch.optim.Adam(list(local_transfer_net.parameters()) + list(refinement_net.parameters()), 
                                     lr=learning_rate, amsgrad=True)
    else:
        optimizer = torch.optim.Adam(list(refinement_net.parameters()), 
                                     lr=learning_rate, amsgrad=True)

In [11]:
def get_random_video_frames(input_videos, frames_num):
    video_id = np.random.randint(0, len(input_videos))
    start_frame = np.random.randint(0, len(input_videos[video_id]) -  1)
    return input_videos[video_id][start_frame: start_frame + frames_num]

In [12]:
def tensor_from_list(list_frames):
    return torch.cat([frame[None, ...] for frame in list_frames], dim=0)

In [13]:
def inference_train_video(frames, refinement_net, local_transfer_net, global_transferer, use_only_local=False):
    I0 = frames[0]
    I_prev = frame_to_tensor(frames[0]).cuda()
    Gk_1 = rgb2gray(I0)
    G0 = Gk_1.copy()
    output_rgb_frames = []
    output_lab_frames = []
    output_local = []
    output_global = []
    for cur_frame in frames[1:]:
        batch = (frame_to_tensor(I0[None, ...]), 
                 I_prev[None, ...].detach(),
                 frame_to_tensor(cur_frame[None, ...]))
        result_lab, result_rgb, result_local, result_global = infer_batch(batch, refinement_net, local_transfer_net, 
                                                                 global_transferer, use_only_local)
        output_rgb_frames.append(result_rgb[0].cpu())
        output_lab_frames.append(result_lab[0])
        output_local.append(result_local[0].cpu())
        output_global.append(result_global[0].cpu())
        I_prev = output_rgb_frames[-1]
        
    return (tensor_from_list(output_lab_frames),
            tensor_from_list(output_rgb_frames), 
            tensor_from_list(output_local), 
            tensor_from_list(output_global))

In [14]:
start_it = 0
inference_frames = 6

In [16]:
start_it = 5000

In [ ]:
#I1, Ik_1, Ik
for it in tqdm(range(start_it, start_it + iteration_num)):
    
    input_frames = get_random_video_frames(train_videos, inference_frames)
    gt_frames = torch.tensor(input_frames).cuda()
    refinement_output_lab_train, refinement_output_rgb_train, local_output_train, global_output_train = inference_train_video(input_frames, refinement_net, 
                                                                                        local_transfer_net, 
                                                                                        global_transferer, 
                                                                                        use_only_local=False)
    
    train_loss = loss_function(refinement_output_lab_train[..., 1:], rgb2lab_torch(gt_frames)[1:, ..., 1:])
    
    
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()
    
    layers_grad = defaultdict()
    
    layer_id = 0
    for i in range(len(refinement_net.module.conv_layers)):
        try:
            weight_grad_mean = torch.mean((refinement_net.module.conv_layers[i].weight.grad) ** 2)
            bias_grad_mean = torch.mean((refinement_net.module.conv_layers[i].bias.grad) ** 2)
            layers_grad["layer_%d_weight_grad_norm" % layer_id] = weight_grad_mean
            layers_grad["layer_%d_bias_grad_norm" % layer_id] = bias_grad_mean
            layer_id += 1
        except:
            continue
    
    
    if (it % verbose_every_it) == 0:
        input_frames = get_random_video_frames(val_videos, inference_frames)
        gt_frames_train = gt_frames[1:]
        gt_frames_val = torch.tensor(input_frames[1:]).cuda()
        refinement_output_lab_val, refinement_output_rgb_val, local_output_val, global_output_val = inference_train_video(input_frames, refinement_net, 
                                                                                        local_transfer_net, 
                                                                                        global_transferer, 
                                                                                        use_only_local=False)
        val_loss = loss_function(refinement_output_lab_val[..., 1:], rgb2lab_torch(gt_frames_val)[..., 1:])
        

        draw_scalar_value(writer, "losses", "train loss", train_loss.detach().cpu().numpy(), it)
        draw_scalar_value(writer, "losses", "val loss", val_loss.detach().cpu().numpy(), it)
        writer.add_scalars("grad_norm", layers_grad, it)
        
        local_output_train = torch.clamp(local_output_train,  0, 1)
        local_output_val = torch.clamp(local_output_val,  0, 1)
        refinement_output_rgb_train = torch.clamp(refinement_output_rgb_train,  0, 1)
        refinement_output_rgb_val = torch.clamp(refinement_output_rgb_val,  0, 1)

        concat_res_train = torch.cat((global_output_train.cpu(), local_output_train.cpu(), refinement_output_rgb_train.cpu()), dim=2)
        draw_images(writer, gt_frames_train.cpu().detach().numpy(), concat_res_train.cpu().detach().numpy(), it, tag="train")
        
        concat_res_val = torch.cat((global_output_val.cpu(), local_output_val.cpu(), refinement_output_rgb_val.cpu()), dim=2)
        draw_images(writer, gt_frames_val.cpu().detach().numpy(), concat_res_val.cpu().detach().numpy(), it, tag="val")
    
    if ((it % save_models_every_it) == 0) and (it > 0):
        os.mkdir(os.path.join(checkpoints_folder, str(it)))
        torch.save(refinement_net, os.path.join(checkpoints_folder,  str(it), "refinement_net"))
        torch.save(refinement_net.state_dict(), os.path.join(checkpoints_folder,  str(it), "refinement_net_state_dict"))
        torch.save(local_transfer_net, os.path.join(checkpoints_folder,  str(it), "local_net"))
        torch.save(local_transfer_net.state_dict(), os.path.join(checkpoints_folder,  str(it), "local_net_state_dict"))
    
    if ((it % test_every_it) == 0) and (it > 0):
        refinement_result, local_result, global_result = inference_all_test_videos(test_videos_folders,
                              refinement_net, local_transfer_net, global_transferer,
                              save_test_output_videos_dir, it, use_only_local=pretrain_local,
                              save_result=True, resize_size=img_shape)
        gt_frames = load_all_test_videos(test_videos_folders, resize_size=img_shape)
        metric_value = calculate_psnr_result(gt_frames, refinement_result)
        draw_scalar_value(writer, "metrics", "test psnr", metric_value, it)
    

        

  0%|          | 0/5000 [00:00<?, ?it/s]
0it [00:00, ?it/s]

  0%|          | 0/90 [00:00<?, ?it/s]

  1%|          | 1/90 [00:04<07:14,  4.88s/it]

  2%|▏         | 2/90 [00:09<07:16,  4.96s/it]

  3%|▎         | 3/90 [00:14<06:52,  4.74s/it]

  4%|▍         | 4/90 [00:18<06:40,  4.66s/it]

  6%|▌         | 5/90 [00:23<06:39,  4.70s/it]

  7%|▋         | 6/90 [00:28<06:35,  4.71s/it]

  8%|▊         | 7/90 [00:32<06:30,  4.70s/it]

  9%|▉         | 8/90 [00:37<06:28,  4.74s/it]

 10%|█         | 9/90 [00:43<06:31,  4.84s/it]

 11%|█         | 10/90 [00:48<06:27,  4.85s/it]

 12%|█▏        | 11/90 [00:53<06:23,  4.85s/it]

 13%|█▎        | 12/90 [00:57<06:16,  4.83s/it]

 14%|█▍        | 13/90 [01:02<06:12,  4.83s/it]

 16%|█▌        | 14/90 [01:07<06:06,  4.82s/it]

 17%|█▋        | 15/90 [01:11<05:58,  4.78s/it]

 18%|█▊        | 16/90 [01:16<05:54,  4.80s/it]

 19%|█▉        | 17/90 [01:21<05:49,  4.79s/it]

 20%|██        | 18/90 [01:25<05:43,  4.78s/it]

 21%|██        | 19/90 [01

  2%|▏         | 1/59 [00:05<04:53,  5.06s/it]

  3%|▎         | 2/59 [00:09<04:29,  4.73s/it]

  5%|▌         | 3/59 [00:14<04:22,  4.70s/it]

  7%|▋         | 4/59 [00:18<04:17,  4.69s/it]

  8%|▊         | 5/59 [00:23<04:11,  4.65s/it]

 10%|█         | 6/59 [00:27<04:06,  4.65s/it]

 12%|█▏        | 7/59 [00:32<04:01,  4.65s/it]

 14%|█▎        | 8/59 [00:37<03:58,  4.68s/it]

 15%|█▌        | 9/59 [00:41<03:52,  4.66s/it]

 17%|█▋        | 10/59 [00:46<03:47,  4.64s/it]

 19%|█▊        | 11/59 [00:50<03:42,  4.64s/it]

 20%|██        | 12/59 [00:55<03:36,  4.61s/it]

 22%|██▏       | 13/59 [00:59<03:32,  4.61s/it]

 24%|██▎       | 14/59 [01:04<03:27,  4.61s/it]

 25%|██▌       | 15/59 [01:09<03:23,  4.62s/it]

 27%|██▋       | 16/59 [01:13<03:18,  4.61s/it]

 29%|██▉       | 17/59 [01:18<03:13,  4.61s/it]

 31%|███       | 18/59 [01:22<03:08,  4.60s/it]

 32%|███▏      | 19/59 [01:27<03:05,  4.63s/it]

 34%|███▍      | 20/59 [01:32<03:00,  4.62s/it]

 36%|███▌      | 21/59 [01:36

 24%|██▍       | 19/79 [01:28<04:38,  4.64s/it]

 25%|██▌       | 20/79 [01:32<04:33,  4.64s/it]

 27%|██▋       | 21/79 [01:37<04:28,  4.63s/it]

 28%|██▊       | 22/79 [01:41<04:23,  4.62s/it]

 29%|██▉       | 23/79 [01:46<04:18,  4.62s/it]

 30%|███       | 24/79 [01:51<04:15,  4.65s/it]

 32%|███▏      | 25/79 [01:56<04:10,  4.64s/it]

 33%|███▎      | 26/79 [02:00<04:05,  4.63s/it]

 34%|███▍      | 27/79 [02:05<04:00,  4.63s/it]

 35%|███▌      | 28/79 [02:09<03:56,  4.63s/it]

 37%|███▋      | 29/79 [02:14<03:51,  4.62s/it]

 38%|███▊      | 30/79 [02:18<03:45,  4.61s/it]

 39%|███▉      | 31/79 [02:22<03:41,  4.61s/it]

 41%|████      | 32/79 [02:27<03:36,  4.61s/it]

 42%|████▏     | 33/79 [02:31<03:31,  4.60s/it]

 43%|████▎     | 34/79 [02:36<03:27,  4.61s/it]

 44%|████▍     | 35/79 [02:41<03:22,  4.61s/it]

 46%|████▌     | 36/79 [02:45<03:18,  4.60s/it]

 47%|████▋     | 37/79 [02:50<03:13,  4.61s/it]

 48%|████▊     | 38/79 [02:54<03:08,  4.60s/it]

 49%|████▉     | 39/

 96%|█████████▌| 86/90 [08:18<00:23,  5.80s/it]

 97%|█████████▋| 87/90 [08:23<00:17,  5.79s/it]

 98%|█████████▊| 88/90 [08:28<00:11,  5.78s/it]

 99%|█████████▉| 89/90 [08:34<00:05,  5.78s/it]

100%|██████████| 90/90 [08:39<00:00,  5.77s/it]


1it [08:40, 520.61s/it]

  0%|          | 0/54 [00:00<?, ?it/s]

  2%|▏         | 1/54 [00:05<04:56,  5.59s/it]

  4%|▎         | 2/54 [00:11<04:52,  5.63s/it]

  6%|▌         | 3/54 [00:16<04:47,  5.63s/it]

  7%|▋         | 4/54 [00:22<04:42,  5.65s/it]

  9%|▉         | 5/54 [00:27<04:32,  5.57s/it]

 11%|█         | 6/54 [00:33<04:28,  5.58s/it]

 13%|█▎        | 7/54 [00:40<04:31,  5.77s/it]

 15%|█▍        | 8/54 [00:46<04:28,  5.83s/it]

 17%|█▋        | 9/54 [00:51<04:18,  5.75s/it]

 19%|█▊        | 10/54 [00:57<04:12,  5.74s/it]

 20%|██        | 11/54 [01:03<04:06,  5.73s/it]

 22%|██▏       | 12/54 [01:08<03:59,  5.71s/it]

 24%|██▍       | 13/54 [01:12<03:49,  5.61s/it]

 26%|██▌       | 14/54 [01:18<03:44,  5.62s/it]

 28%|██▊    

 41%|████      | 28/69 [02:39<03:54,  5.71s/it]

 42%|████▏     | 29/69 [02:45<03:47,  5.70s/it]

 43%|████▎     | 30/69 [02:50<03:41,  5.67s/it]

 45%|████▍     | 31/69 [02:55<03:35,  5.66s/it]

 46%|████▋     | 32/69 [03:01<03:29,  5.67s/it]

 48%|████▊     | 33/69 [03:06<03:23,  5.66s/it]

 49%|████▉     | 34/69 [03:12<03:17,  5.65s/it]

 51%|█████     | 35/69 [03:17<03:12,  5.65s/it]

 52%|█████▏    | 36/69 [03:25<03:08,  5.70s/it]

 54%|█████▎    | 37/69 [03:33<03:04,  5.77s/it]

 55%|█████▌    | 38/69 [03:42<03:01,  5.85s/it]

 57%|█████▋    | 39/69 [03:50<02:57,  5.92s/it]

 58%|█████▊    | 40/69 [04:00<02:54,  6.01s/it]

 59%|█████▉    | 41/69 [04:08<02:49,  6.05s/it]

 61%|██████    | 42/69 [04:13<02:43,  6.04s/it]

 62%|██████▏   | 43/69 [04:18<02:36,  6.02s/it]

 64%|██████▍   | 44/69 [04:24<02:30,  6.01s/it]

 65%|██████▌   | 45/69 [04:29<02:23,  5.98s/it]

 67%|██████▋   | 46/69 [04:34<02:17,  5.97s/it]

 68%|██████▊   | 47/69 [04:39<02:10,  5.95s/it]

 70%|██████▉   | 48/

...

 80%|████████  | 72/90 [06:48<01:42,  5.67s/it]

 81%|████████  | 73/90 [06:53<01:36,  5.66s/it]

 82%|████████▏ | 74/90 [06:57<01:30,  5.65s/it]

 83%|████████▎ | 75/90 [07:03<01:24,  5.65s/it]

 84%|████████▍ | 76/90 [07:08<01:18,  5.63s/it]

 86%|████████▌ | 77/90 [07:13<01:13,  5.63s/it]

 87%|████████▋ | 78/90 [07:18<01:07,  5.62s/it]

 88%|████████▊ | 79/90 [07:23<01:01,  5.62s/it]

 89%|████████▉ | 80/90 [07:29<00:56,  5.61s/it]

 90%|█████████ | 81/90 [07:35<00:50,  5.63s/it]

 91%|█████████ | 82/90 [07:40<00:44,  5.62s/it]

 92%|█████████▏| 83/90 [07:46<00:39,  5.62s/it]

 93%|█████████▎| 84/90 [07:51<00:33,  5.62s/it]

 94%|█████████▍| 85/90 [07:56<00:28,  5.61s/it]

 96%|█████████▌| 86/90 [08:03<00:22,  5.62s/it]

 97%|█████████▋| 87/90 [08:08<00:16,  5.62s/it]

 98%|█████████▊| 88/90 [08:15<00:11,  5.63s/it]

 99%|█████████▉| 89/90 [08:21<00:05,  5.64s/it]

100%|██████████| 90/90 [08:29<00:00,  5.66s/it]


1it [08:32, 512.03s/it]

  0%|          | 0/54 [00:00<?, ?it/s]

  2

 20%|██        | 14/69 [01:26<05:38,  6.16s/it]

 22%|██▏       | 15/69 [01:32<05:33,  6.18s/it]

 23%|██▎       | 16/69 [01:38<05:25,  6.14s/it]

 25%|██▍       | 17/69 [01:45<05:21,  6.18s/it]

 26%|██▌       | 18/69 [01:50<05:12,  6.12s/it]

 28%|██▊       | 19/69 [01:56<05:05,  6.12s/it]

 29%|██▉       | 20/69 [02:01<04:58,  6.10s/it]

 30%|███       | 21/69 [02:08<04:53,  6.11s/it]

 32%|███▏      | 22/69 [02:13<04:44,  6.05s/it]

 33%|███▎      | 23/69 [02:18<04:37,  6.04s/it]

 35%|███▍      | 24/69 [02:25<04:32,  6.06s/it]

 36%|███▌      | 25/69 [02:31<04:27,  6.07s/it]

 38%|███▊      | 26/69 [02:37<04:20,  6.06s/it]

 39%|███▉      | 27/69 [02:44<04:16,  6.10s/it]

 41%|████      | 28/69 [02:50<04:09,  6.08s/it]

 42%|████▏     | 29/69 [02:56<04:03,  6.08s/it]

 43%|████▎     | 30/69 [03:01<03:55,  6.05s/it]

 45%|████▍     | 31/69 [03:07<03:49,  6.04s/it]

 46%|████▋     | 32/69 [03:12<03:43,  6.03s/it]

 48%|████▊     | 33/69 [03:18<03:36,  6.02s/it]

 49%|████▉     | 34/

  7%|▋         | 6/90 [00:38<09:00,  6.44s/it]

  8%|▊         | 7/90 [00:42<08:29,  6.14s/it]

  9%|▉         | 8/90 [00:47<08:09,  5.97s/it]

 10%|█         | 9/90 [00:52<07:55,  5.87s/it]

 11%|█         | 10/90 [01:01<08:14,  6.18s/it]

 12%|█▏        | 11/90 [01:08<08:08,  6.19s/it]

 13%|█▎        | 12/90 [01:13<08:00,  6.16s/it]

 14%|█▍        | 13/90 [01:19<07:48,  6.08s/it]

 16%|█▌        | 14/90 [01:25<07:43,  6.10s/it]

 17%|█▋        | 15/90 [01:32<07:44,  6.20s/it]

 18%|█▊        | 16/90 [01:41<07:51,  6.37s/it]

 19%|█▉        | 17/90 [01:47<07:42,  6.34s/it]

 20%|██        | 18/90 [01:54<07:39,  6.39s/it]

 21%|██        | 19/90 [02:00<07:31,  6.36s/it]

 22%|██▏       | 20/90 [02:06<07:22,  6.32s/it]

 23%|██▎       | 21/90 [02:11<07:11,  6.25s/it]

 24%|██▍       | 22/90 [02:18<07:06,  6.28s/it]

 26%|██▌       | 23/90 [02:23<06:58,  6.25s/it]

 27%|██▋       | 24/90 [02:28<06:49,  6.20s/it]

 28%|██▊       | 25/90 [02:34<06:41,  6.18s/it]

 29%|██▉       | 26/90 [

 14%|█▎        | 8/59 [00:45<04:51,  5.72s/it]

 15%|█▌        | 9/59 [00:50<04:42,  5.64s/it]

 17%|█▋        | 10/59 [00:55<04:33,  5.59s/it]

 19%|█▊        | 11/59 [01:01<04:27,  5.58s/it]

 20%|██        | 12/59 [01:07<04:22,  5.58s/it]

 22%|██▏       | 13/59 [01:12<04:15,  5.55s/it]

 24%|██▎       | 14/59 [01:17<04:10,  5.56s/it]

 25%|██▌       | 15/59 [01:24<04:06,  5.61s/it]

 27%|██▋       | 16/59 [01:30<04:01,  5.63s/it]

 29%|██▉       | 17/59 [01:34<03:54,  5.59s/it]

 31%|███       | 18/59 [01:42<03:52,  5.68s/it]

 32%|███▏      | 19/59 [01:48<03:47,  5.70s/it]

 34%|███▍      | 20/59 [01:54<03:42,  5.72s/it]

 36%|███▌      | 21/59 [01:58<03:35,  5.66s/it]

 37%|███▋      | 22/59 [02:04<03:29,  5.65s/it]

 39%|███▉      | 23/59 [02:10<03:24,  5.67s/it]

 41%|████      | 24/59 [02:15<03:17,  5.63s/it]

 42%|████▏     | 25/59 [02:19<03:10,  5.59s/it]

 44%|████▍     | 26/59 [02:25<03:05,  5.61s/it]

 46%|████▌     | 27/59 [02:32<03:00,  5.65s/it]

 47%|████▋     | 28/59

 33%|███▎      | 26/79 [02:31<05:08,  5.82s/it]

 34%|███▍      | 27/79 [02:37<05:02,  5.82s/it]

 35%|███▌      | 28/79 [02:42<04:55,  5.79s/it]

 37%|███▋      | 29/79 [02:48<04:50,  5.81s/it]

 38%|███▊      | 30/79 [02:54<04:45,  5.82s/it]

 39%|███▉      | 31/79 [03:00<04:39,  5.83s/it]

 41%|████      | 32/79 [03:06<04:33,  5.82s/it]

 42%|████▏     | 33/79 [03:12<04:28,  5.84s/it]

 43%|████▎     | 34/79 [03:18<04:22,  5.84s/it]

 44%|████▍     | 35/79 [03:25<04:18,  5.87s/it]

 46%|████▌     | 36/79 [03:30<04:11,  5.84s/it]

 47%|████▋     | 37/79 [03:36<04:05,  5.84s/it]

 48%|████▊     | 38/79 [03:43<04:00,  5.87s/it]

 49%|████▉     | 39/79 [03:48<03:54,  5.86s/it]

 51%|█████     | 40/79 [03:53<03:47,  5.84s/it]

 52%|█████▏    | 41/79 [04:00<03:42,  5.87s/it]

 53%|█████▎    | 42/79 [04:06<03:36,  5.86s/it]

 54%|█████▍    | 43/79 [04:11<03:30,  5.85s/it]

 56%|█████▌    | 44/79 [04:16<03:24,  5.84s/it]

 57%|█████▋    | 45/79 [04:23<03:19,  5.86s/it]

 58%|█████▊    | 46/

In [ ]:
videos_frames = load_all_test_videos(test_videos_folders, img_shape)